In [ ]:
import torch
import torch.nn.functional as F

# 예시: 예측 시퀀스 (로그 확률)
# batch_size=1, num_classes=5 (a, b, c, d, blank), seq_len=10
# (time, batch, num_classes)
# 'a', 'a', 'a', 'b', 'b', 'b', blank, 'c', 'c', 'c'
# 실제 CTC 모델의 출력은 소프트맥스 후 로그 확률 형태입니다.
# 여기서는 간단히 표현했습니다.
log_probs = torch.full((10, 1, 5), -10.0) # 모든 클래스에 대해 낮은 확률
log_probs[0, 0, 0] = 0.0 # 'a'
log_probs[1, 0, 0] = 0.0 # 'a'
log_probs[2, 0, 0] = 0.0 # 'a'
log_probs[3, 0, 1] = 0.0 # 'b'
log_probs[4, 0, 1] = 0.0 # 'b'
log_probs[5, 0, 1] = 0.0 # 'b'
log_probs[6, 0, 4] = 0.0 # blank (index 4)
log_probs[7, 0, 2] = 0.0 # 'c'
log_probs[8, 0, 2] = 0.0 # 'c'
log_probs[9, 0, 2] = 0.0 # 'c'

# 목표 레이블 (정답 시퀀스)
# 'a', 'b', 'c'
# CTC에서 blank 토큰은 레이블에 포함되지 않습니다.
# 인덱스는 클래스 인덱스와 일치해야 합니다. (0:a, 1:b, 2:c, 3:d)
targets = torch.tensor([0, 1, 2], dtype=torch.long)

# 예측 시퀀스의 길이
input_lengths = torch.tensor([10], dtype=torch.long)
# 목표 시퀀스의 길이
target_lengths = torch.tensor([3], dtype=torch.long)

# CTCLoss 초기화 (blank는 마지막 클래스 인덱스로 가정, default=0)
# PyTorch의 CTCLoss는 blank를 num_classes-1로 가정합니다.
# 따라서 위의 예시에서 num_classes=5일 때 blank는 인덱스 4입니다.
ctc_loss = torch.nn.CTCLoss(blank=4, reduction='mean')

# 손실 계산
loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
print(f"CTC Loss: {loss.item()}")

# 디코딩 예시 (가장 확률 높은 경로)
# 이 과정은 실제로는 훨씬 복잡한 빔 서치(Beam Search) 등을 포함합니다.
# 여기서는 단순히 가장 높은 확률의 토큰을 선택하고 압축하는 방식으로 시뮬레이션합니다.
output_indices = torch.argmax(log_probs.squeeze(1), dim=1)
print(f"Predicted indices (raw): {output_indices}")

# CTC 디코딩 규칙 적용: 연속된 동일 토큰 압축 및 blank 제거
decoded_chars = []
last_char = -1
for char_idx in output_indices:
    if char_idx != 4 and char_idx != last_char: # blank가 아니고, 이전과 다른 문자일 경우
        decoded_chars.append(chr(ord('a') + char_idx.item()))
    last_char = char_idx

print(f"Decoded sequence: {''.join(decoded_chars)}")

CTC Loss: -0.00015139028255362064
Predicted indices (raw): tensor([0, 0, 0, 1, 1, 1, 4, 2, 2, 2])
Decoded sequence: abc


attention 기반 ASR

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 가상의 인코더 출력 (소스 시퀀스)
# (batch_size, seq_len, hidden_dim)
encoder_outputs = torch.randn(1, 10, 128) # 10개의 타임스텝, 각 128차원

# 가상의 디코더 상태 (쿼리)
# (batch_size, hidden_dim)
decoder_state = torch.randn(1, 128) # 현재 디코더 스텝의 상태

# 간단한 어텐션 레이어 (Dot-product attention)
class SimpleAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.query_proj = nn.Linear(hidden_dim, hidden_dim)
        self.key_proj = nn.Linear(hidden_dim, hidden_dim)
        self.value_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, query, keys, values):
        # 쿼리, 키, 값 변환
        query_projected = self.query_proj(query).unsqueeze(1) # (batch, 1, hidden_dim)
        keys_projected = self.key_proj(keys) # (batch, seq_len, hidden_dim)
        values_projected = self.value_proj(values) # (batch, seq_len, hidden_dim)

        # 어텐션 스코어 계산 (쿼리 * 키.T)
        # (batch, 1, hidden_dim) @ (batch, hidden_dim, seq_len) -> (batch, 1, seq_len)
        attention_scores = torch.bmm(query_projected, keys_projected.transpose(1, 2))

        # 소프트맥스를 적용하여 어텐션 가중치 (확률 분포) 얻기
        attention_weights = F.softmax(attention_scores, dim=-1) # (batch, 1, seq_len)

        # 어텐션 가중치와 값 곱하여 문맥 벡터 생성
        # (batch, 1, seq_len) @ (batch, seq_len, hidden_dim) -> (batch, 1, hidden_dim)
        context_vector = torch.bmm(attention_weights, values_projected)
        return context_vector.squeeze(1), attention_weights.squeeze(1)

attention_module = SimpleAttention(128)
context_vector, attention_weights = attention_module(decoder_state, encoder_outputs, encoder_outputs)

print(f"Decoder State (Query) shape: {decoder_state.shape}")
print(f"Encoder Outputs (Keys/Values) shape: {encoder_outputs.shape}")
print(f"Context Vector shape: {context_vector.shape}")
print(f"Attention Weights shape (sum to 1 for each query): {attention_weights.shape}")
print(f"Example Attention Weights (first batch): {attention_weights[0].tolist()}")

# 이 문맥 벡터는 디코더의 다음 예측을 위한 입력으로 사용됩니다.

Decoder State (Query) shape: torch.Size([1, 128])
Encoder Outputs (Keys/Values) shape: torch.Size([1, 10, 128])
Context Vector shape: torch.Size([1, 128])
Attention Weights shape (sum to 1 for each query): torch.Size([1, 10])
Example Attention Weights (first batch): [0.0012099185260012746, 8.871893805917352e-05, 0.01581745408475399, 1.4953170648368541e-05, 9.84927737590624e-06, 0.002038335893303156, 0.0024939654394984245, 8.627141028227925e-07, 0.9776942729949951, 0.0006317192455753684]


최신 ASR 모델 사용 예시 코드

In [ ]:
# 필요한 패키지 설치
# !pip install transformers accelerate soundfile librosa

import warnings
warnings.filterwarnings("ignore")

import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import soundfile as sf
import librosa
import numpy as np
from base64 import b64decode
from google.colab.output import eval_js
from IPython.display import display
import time

# 1) 모델과 프로세서 로드
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2) 파일 단위 인식 함수
def transcribe_file(path): #음성 파일을 입력받아 그것을 출력하도록 하는 함수
    try:
        audio, sr = sf.read(path)
    except:
        audio, sr = librosa.load(path, sr=16000)  #wave form내 8kHz 안에 있는 data 보겠다.
    inputs = processor(audio, sampling_rate=sr, return_tensors="pt", padding=True)
    input_features = inputs.input_features.to(device)

    # Whisper가 기대하는 입력 길이 계산(최대 30s)
    encoder = model.model.encoder
    stride1 = encoder.conv1.stride[0]
    stride2 = encoder.conv2.stride[0]
    expected_len = model.config.max_source_positions * stride1 * stride2

    seq_len = input_features.shape[-1]
    if seq_len < expected_len:
        pad_len = expected_len - seq_len
        input_features = torch.nn.functional.pad(input_features, (0, pad_len))
    elif seq_len > expected_len:
        input_features = input_features[..., :expected_len]

    # attention_mask 생성
    attention_mask = torch.ones(input_features.shape[:-1], device=device)

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features=input_features,
            attention_mask=attention_mask
        )
    return processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# 3) 마이크 권한 요청
print("마이크 권한 요청 중…")
eval_js("""
  navigator.mediaDevices.getUserMedia({ audio: true })
    .then(stream => stream.getTracks().forEach(t => t.stop()))
    .catch(e => console.error(e));
""")

# 4) 녹음 및 인식 함수 (UI 개선)
def record_and_transcribe_ui(filename="recorded_audio.webm", duration=3):
    status = display("준비 중", display_id=True)
    status.update("🔴 녹음 중")
    js = f"""
    async function recordAudio() {{
      const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
      const recorder = new MediaRecorder(stream);
      const chunks = [];
      recorder.ondataavailable = e => chunks.push(e.data);
      recorder.start();
      await new Promise(r => setTimeout(r, {duration * 1000}));
      recorder.stop();
      await new Promise(r => recorder.onstop = r);
      const blob = new Blob(chunks);
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      return await new Promise(res => reader.onloadend = () => res(reader.result.split(',')[1]));
    }}
    recordAudio();
    """
    try:
        b64data = eval_js(js)
        audio_bytes = b64decode(b64data)
        with open(filename, "wb") as f:
            f.write(audio_bytes)

        # 인식 진행 표시
        for i in range(6):
            status.update("⌛ 인식 중" + "." * ((i % 3) + 1))
            time.sleep(0.5)

        result = transcribe_file(filename)
        status.update(f"✅ 인식 결과: {result}")
        return result

    except Exception as e:
        status.update(f"⚠️ 오류 발생: {e}")
        raise


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

마이크 권한 요청 중…


In [ ]:
def levenshtein_ops(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = [[0]*(n+1) for _ in range(m+1)]
    op = [[None]*(n+1) for _ in range(m+1)]  # 연산 기록

    for i in range(m+1):
        dp[i][0] = i
        op[i][0] = 'D' if i > 0 else None
    for j in range(n+1):
        dp[0][j] = j
        op[0][j] = 'I' if j > 0 else None

    for i in range(1, m+1):
        for j in range(1, n+1):
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1]
                op[i][j] = 'E'  # 일치
            else:
                del_cost = dp[i-1][j] + 1
                ins_cost = dp[i][j-1] + 1
                sub_cost = dp[i-1][j-1] + 1
                min_cost = min(del_cost, ins_cost, sub_cost)
                dp[i][j] = min_cost
                if min_cost == sub_cost:
                    op[i][j] = 'S'  # 치환
                elif min_cost == del_cost:
                    op[i][j] = 'D'  # 삭제
                else:
                    op[i][j] = 'I'  # 삽입

    # 연산 추적
    i, j = m, n
    subs, ins, dels = 0, 0, 0
    while i > 0 or j > 0:
        if op[i][j] == 'E':
            i -= 1
            j -= 1
        elif op[i][j] == 'S':
            subs += 1
            i -= 1
            j -= 1
        elif op[i][j] == 'D':
            dels += 1
            i -= 1
        elif op[i][j] == 'I':
            ins += 1
            j -= 1

    return dp[m][n], subs, ins, dels

# -------------------------------
# 평가 함수 (WER, CER + 정확도 + 세부연산)
# -------------------------------

def wer_detail(reference, hypothesis):
    ref_words = reference.strip().split()
    hyp_words = hypothesis.strip().split()
    distance, subs, ins, dels = levenshtein_ops(ref_words, hyp_words)
    wer_score = distance / max(len(ref_words), 1)
    accuracy = (len(ref_words) - distance) / max(len(ref_words), 1)
    return wer_score, accuracy, subs, ins, dels

def cer_detail(reference, hypothesis):
    ref_chars = list(reference.strip().replace(" ", ""))
    hyp_chars = list(hypothesis.strip().replace(" ", ""))
    distance, subs, ins, dels = levenshtein_ops(ref_chars, hyp_chars)
    cer_score = distance / max(len(ref_chars), 1)
    accuracy = (len(ref_chars) - distance) / max(len(ref_chars), 1)
    return cer_score, accuracy, subs, ins, dels

# -------------------------------
# 예제 입력 및 출력
# -------------------------------
index=234
my_list=[
      "안녕하세요, 오늘 날씨가 정말 좋네요. 산책하기 딱 좋은 날씨예요.",
      "인공지능 기술은 빠르게 발전하고 있으며, 우리의 삶에 많은 변화를 가져오고 있습니다.",
      "아, 이런! 깜빡하고 우산을 놓고 왔네. 소나기가 올 수도 있다고 했는데.",
      "대한민국의 수도 서울은 다채로운 역사와 현대적인 매력이 공존하는 도시입니다.",
      "혹시 파이썬으로 텐서플로우나 파이토치를 사용해 보신 적이 있으신가요?",
      "칠월 칠석은 견우와 직녀가 일 년에 한 번 만나는 날이라고 전해집니다.",
      "지금 들으시는 음악은 바흐의 평균율 클라비어 곡집 중 한 곡입니다.",
      "죄송합니다, 잠시 통신 상태가 좋지 않아 말씀이 잘 들리지 않습니다.",
      "복잡한 재무제표를 분석하여 기업의 건전성을 평가하는 것은 매우 중요합니다.",
      "음, 글쎄요. 정확한 답변을 드리기 위해서는 좀 더 많은 정보가 필요할 것 같습니다."
      ]

for i in range(10):
  hyp_text=record_and_transcribe_ui(duration=6)
  ref_text=my_list[i]

  print("Reference :", ref_text)
  print("Hypothesis:", hyp_text)

  # WER
  wer_score, wer_acc, wer_s, wer_i, wer_d = wer_detail(ref_text, hyp_text)
  print("\n[WER 평가 - 단어 단위]")
  print("WER (오류율)      :", round(wer_score, 3))
  print("정확도 (Accuracy) :", round(wer_acc, 3))
  print("치환(Sub)         :", wer_s)
  print("삽입(Ins)         :", wer_i)
  print("삭제(Del)         :", wer_d)

  # CER
  cer_score, cer_acc, cer_s, cer_i, cer_d = cer_detail(ref_text, hyp_text)
  print("\n[CER 평가 - 문자 단위]")
  print("CER (오류율)      :", round(cer_score, 3))
  print("정확도 (Accuracy) :", round(cer_acc, 3))
  print("치환(Sub)         :", cer_s)
  print("삽입(Ins)         :", cer_i)
  print("삭제(Del)         :", cer_d)

'✅ 인식 결과:  안녕하세요 오늘 날씨가 정말 좋네요 산책하기 딱 좋은 날씨에요'

Reference : 안녕하세요, 오늘 날씨가 정말 좋네요. 산책하기 딱 좋은 날씨예요.
Hypothesis:  안녕하세요 오늘 날씨가 정말 좋네요 산책하기 딱 좋은 날씨에요

[WER 평가 - 단어 단위]
WER (오류율)      : 0.333
정확도 (Accuracy) : 0.667
치환(Sub)         : 3
삽입(Ins)         : 0
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오류율)      : 0.138
정확도 (Accuracy) : 0.862
치환(Sub)         : 1
삽입(Ins)         : 0
삭제(Del)         : 3


'✅ 인식 결과:  인공지능 기술은 빠르게 발전하고 있어요. 우리 삶의 많은 변화를 가져오고 있습니다.'

Reference : 인공지능 기술은 빠르게 발전하고 있으며, 우리의 삶에 많은 변화를 가져오고 있습니다.
Hypothesis:  인공지능 기술은 빠르게 발전하고 있어요. 우리 삶의 많은 변화를 가져오고 있습니다.

[WER 평가 - 단어 단위]
WER (오류율)      : 0.273
정확도 (Accuracy) : 0.727
치환(Sub)         : 3
삽입(Ins)         : 0
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오류율)      : 0.135
정확도 (Accuracy) : 0.865
치환(Sub)         : 4
삽입(Ins)         : 0
삭제(Del)         : 1


'✅ 인식 결과:  아 이런 깜빡하고 우선을 놓고 왔네. 송하기가 올 수 있다고 했는데'

...

Reference : 죄송합니다, 잠시 통신 상태가 좋지 않아 말씀이 잘 들리지 않습니다.
Hypothesis:  죄송합니다. 잠시 통신 상태가 주시지 않았던 말씀이 잘 들리지 않습니다.

[WER 평가 - 단어 단위]
WER (오류율)      : 0.3
정확도 (Accuracy) : 0.7
치환(Sub)         : 3
삽입(Ins)         : 0
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오류율)      : 0.172
정확도 (Accuracy) : 0.828
치환(Sub)         : 3
삽입(Ins)         : 2
삭제(Del)         : 0


'✅ 인식 결과:  복잡한 제모 제표를 변석하여 기업의 건정성을 평가하는 것은 매우 중요합니다.'

Reference : 복잡한 재무제표를 분석하여 기업의 건전성을 평가하는 것은 매우 중요합니다.
Hypothesis:  복잡한 제모 제표를 변석하여 기업의 건정성을 평가하는 것은 매우 중요합니다.

[WER 평가 - 단어 단위]
WER (오류율)      : 0.444
정확도 (Accuracy) : 0.556
치환(Sub)         : 3
삽입(Ins)         : 1
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오류율)      : 0.121
정확도 (Accuracy) : 0.879
치환(Sub)         : 4
삽입(Ins)         : 0
삭제(Del)         : 0


'✅ 인식 결과:  음 글쎄요 정확한 답변을 드리기 위해서는 좀 더 많은 정보가 필요할'

Reference : 음, 글쎄요. 정확한 답변을 드리기 위해서는 좀 더 많은 정보가 필요할 것 같습니다.
Hypothesis:  음 글쎄요 정확한 답변을 드리기 위해서는 좀 더 많은 정보가 필요할

[WER 평가 - 단어 단위]
WER (오류율)      : 0.308
정확도 (Accuracy) : 0.692
치환(Sub)         : 2
삽입(Ins)         : 0
삭제(Del)         : 2

[CER 평가 - 문자 단위]
CER (오류율)      : 0.229
정확도 (Accuracy) : 0.771
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 8


실습

In [6]:
import base64
import time
import soundfile as sf
from google.colab import output

RECORD_SEC = 2  # 실제 녹음 시간

# JavaScript 녹음 코드
recording_code = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = () => resolve(reader.result.split(',')[1]);
  reader.readAsDataURL(blob);
});

async function record(sec=3) {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const mediaRecorder = new MediaRecorder(stream);
  const audioChunks = [];

  mediaRecorder.ondataavailable = e => audioChunks.push(e.data);
  mediaRecorder.start();

  await sleep(sec * 1000);
  mediaRecorder.stop();

  await new Promise(resolve => mediaRecorder.onstop = resolve);
  const audioBlob = new Blob(audioChunks);
  const base64data = await b2text(audioBlob);
  return base64data;
}
record();
"""

# 개선된 녹음 함수
def record_audio_progress(filename="recorded.wav"):
    print("녹음을 곧 시작합니다. 준비하세요.")
    for i in range(1, 0, -1):
        print(f"{i}...", end='', flush=True)
        time.sleep(1)
    print("\n● 녹음 중...", flush=True)

    recorded = output.eval_js(recording_code)

    audio_bytes = base64.b64decode(recorded)
    with open(filename, "wb") as f:
        f.write(audio_bytes)

    print("녹음 완료 및 저장:", filename)
    return filename

In [4]:
def record_multiple_audios(base_filename="word", count=10):
    file_paths = []

    for i in range(count):
        filename = f"{base_filename}_{i+1}.wav"
        print(f"\n[{i+1}/{count}] 파일명: {filename}")
        path = record_audio_progress(filename)
        file_paths.append(path)

    print("\n총 녹음 완료 파일:")
    for f in file_paths:
        print(" -", f)

    return file_paths

In [7]:
file_paths = record_multiple_audios(base_filename="word", count=100)


[1/100] 파일명: word_1.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_1.wav

[2/100] 파일명: word_2.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_2.wav

[3/100] 파일명: word_3.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_3.wav

[4/100] 파일명: word_4.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_4.wav

[5/100] 파일명: word_5.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_5.wav

[6/100] 파일명: word_6.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_6.wav

[7/100] 파일명: word_7.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_7.wav

[8/100] 파일명: word_8.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_8.wav

[9/100] 파일명: word_9.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_9.wav

[10/100] 파일명: word_10.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_10.wav

[11/100] 파일명: word_11.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_11.wav

[12/100] 파일명: word_12.wav
녹음을 곧 시작합니다. 준비하세요.
1...
● 녹음 중...
녹음 완료 및 저장: word_12

In [8]:
!pip install hgtk

In [9]:
#한국어 자모 분리
import hgtk

def split_jamos(text):
    result = []
    for ch in text:
        if hgtk.checker.is_hangul(ch):
            chosung, jungsung, jongsung = hgtk.letter.decompose(ch)
            result.append(chosung)
            result.append(jungsung)
            if jongsung != '':
                result.append(jongsung)
        else:
            result.append(ch)
    return result

In [11]:
#한국어 딕셔너리 구성
label_dict = {
    "word_1.wav": split_jamos("도와줘"),
    "word_2.wav": split_jamos("도와줘"),
    "word_3.wav": split_jamos("도와줘"),
    "word_4.wav": split_jamos("도와줘"),
    "word_5.wav": split_jamos("도와줘"),
    "word_6.wav": split_jamos("도와줘"),
    "word_7.wav": split_jamos("도와줘"),
    "word_8.wav": split_jamos("도와줘"),
    "word_9.wav": split_jamos("도와줘"),
    "word_10.wav": split_jamos("도와줘"),
    "word_11.wav": split_jamos("구급차 불러"),
    "word_12.wav": split_jamos("구급차 불러"),
    "word_13.wav": split_jamos("구급차 불러"),
    "word_14.wav": split_jamos("구급차 불러"),
    "word_15.wav": split_jamos("구급차 불러"),
    "word_16.wav": split_jamos("구급차 불러"),
    "word_17.wav": split_jamos("구급차 불러"),
    "word_18.wav": split_jamos("구급차 불러"),
    "word_19.wav": split_jamos("구급차 불러"),
    "word_20.wav": split_jamos("구급차 불러"),
    "word_21.wav": split_jamos("불났어"),
    "word_22.wav": split_jamos("불났어"),
    "word_23.wav": split_jamos("불났어"),
    "word_24.wav": split_jamos("불났어"),
    "word_25.wav": split_jamos("불났어"),
    "word_26.wav": split_jamos("불났어"),
    "word_27.wav": split_jamos("불났어"),
    "word_28.wav": split_jamos("불났어"),
    "word_29.wav": split_jamos("불났어"),
    "word_30.wav": split_jamos("불났어"),
    "word_31.wav": split_jamos("쓰러졌어"),
    "word_32.wav": split_jamos("쓰러졌어"),
    "word_33.wav": split_jamos("쓰러졌어"),
    "word_34.wav": split_jamos("쓰러졌어"),
    "word_35.wav": split_jamos("쓰러졌어"),
    "word_36.wav": split_jamos("쓰러졌어"),
    "word_37.wav": split_jamos("쓰러졌어"),
    "word_38.wav": split_jamos("쓰러졌어"),
    "word_39.wav": split_jamos("쓰러졌어"),
    "word_40.wav": split_jamos("쓰러졌어"),
    "word_41.wav": split_jamos("위험해"),
    "word_42.wav": split_jamos("위험해"),
    "word_43.wav": split_jamos("위험해"),
    "word_44.wav": split_jamos("위험해"),
    "word_45.wav": split_jamos("위험해"),
    "word_46.wav": split_jamos("위험해"),
    "word_47.wav": split_jamos("위험해"),
    "word_48.wav": split_jamos("위험해"),
    "word_49.wav": split_jamos("위험해"),
    "word_50.wav": split_jamos("위험해"),
    "word_51.wav": split_jamos("경찰불러"),
    "word_52.wav": split_jamos("경찰불러"),
    "word_53.wav": split_jamos("경찰불러"),
    "word_54.wav": split_jamos("경찰불러"),
    "word_55.wav": split_jamos("경찰불러"),
    "word_56.wav": split_jamos("경찰불러"),
    "word_57.wav": split_jamos("경찰불러"),
    "word_58.wav": split_jamos("경찰불러"),
    "word_59.wav": split_jamos("경찰불러"),
    "word_60.wav": split_jamos("경찰불러"),
    "word_61.wav": split_jamos("아파"),
    "word_62.wav": split_jamos("아파"),
    "word_63.wav": split_jamos("아파"),
    "word_64.wav": split_jamos("아파"),
    "word_65.wav": split_jamos("아파"),
    "word_66.wav": split_jamos("아파"),
    "word_67.wav": split_jamos("아파"),
    "word_68.wav": split_jamos("아파"),
    "word_69.wav": split_jamos("아파"),
    "word_70.wav": split_jamos("아파"),
    "word_71.wav": split_jamos("구조요청"),
    "word_72.wav": split_jamos("구조요청"),
    "word_73.wav": split_jamos("구조요청"),
    "word_74.wav": split_jamos("구조요청"),
    "word_75.wav": split_jamos("구조요청"),
    "word_76.wav": split_jamos("구조요청"),
    "word_77.wav": split_jamos("구조요청"),
    "word_78.wav": split_jamos("구조요청"),
    "word_79.wav": split_jamos("구조요청"),
    "word_80.wav": split_jamos("구조요청"),
    "word_81.wav": split_jamos("약가져와"),
    "word_82.wav": split_jamos("약가져와"),
    "word_83.wav": split_jamos("약가져와"),
    "word_84.wav": split_jamos("약가져와"),
    "word_85.wav": split_jamos("약가져와"),
    "word_86.wav": split_jamos("약가져와"),
    "word_87.wav": split_jamos("약가져와"),
    "word_88.wav": split_jamos("약가져와"),
    "word_89.wav": split_jamos("약가져와"),
    "word_90.wav": split_jamos("약가져와"),
    "word_91.wav": split_jamos("응급모드시작작"),
    "word_92.wav": split_jamos("응급모드시작작"),
    "word_93.wav": split_jamos("응급모드시작작"),
    "word_94.wav": split_jamos("응급모드시작작"),
    "word_95.wav": split_jamos("응급모드시작작"),
    "word_96.wav": split_jamos("응급모드시작작"),
    "word_97.wav": split_jamos("응급모드시작작"),
    "word_98.wav": split_jamos("응급모드시작작"),
    "word_99.wav": split_jamos("응급모드시작작"),
    "word_100.wav": split_jamos("응급모드시작작"),
}

In [12]:
import torch
import torchaudio
import librosa
import numpy as np

vocab = sorted(set(c for lbl in label_dict.values() for c in lbl))
char2idx = {c: i + 1 for i, c in enumerate(vocab)}  # 0 = blank
idx2char = {i: c for c, i in char2idx.items()}
num_classes = len(char2idx) + 1  # CTC blank 포함

In [13]:
def extract_mfcc_and_label(path, label_str, max_len=150):
    y, sr = librosa.load(path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc = mfcc.T  # [T, 13]

    if mfcc.shape[0] < max_len:
        pad = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad), (0, 0)), mode='constant')
    else:
        mfcc = mfcc[:max_len]

    label_ids = torch.tensor([char2idx[c] for c in label_str], dtype=torch.long)
    return torch.tensor(mfcc, dtype=torch.float32), label_ids

In [14]:
features, targets = [], []
input_lengths, target_lengths = [], []

for fname, label in label_dict.items():
    mfcc, label_tensor = extract_mfcc_and_label(fname, label)
    features.append(mfcc)
    targets.append(label_tensor)
    input_lengths.append(mfcc.shape[0])
    target_lengths.append(len(label_tensor))

features = torch.stack(features)                    # [B, T, D]
targets = torch.cat(targets)                        # 1D targets for CTC
input_lengths = torch.tensor(input_lengths)         # [B]
target_lengths = torch.tensor(target_lengths)       # [B]

/tmp/ipython-input-13-2675425067.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-13-2675425067.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-13-2675425067.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/cor

In [15]:
import torch.nn as nn

class CTCASRModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(128 * 2, num_classes)  # bidirectional

    def forward(self, x):  # x: [B, T, D]
        out, _ = self.lstm(x)
        out = self.fc(out)  # [B, T, num_classes]
        return out.log_softmax(2)  # CTC용 log_softmax

In [16]:
model = CTCASRModel(input_dim=13, num_classes=num_classes)
ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(features)                    # [B, T, C]
    output = output.transpose(0, 1)             # [T, B, C] for CTCLoss
    loss = ctc_loss(output, targets, input_lengths, target_lengths)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 55.5110
Epoch 2, Loss: 51.4637
Epoch 3, Loss: 46.5356
Epoch 4, Loss: 40.9519
Epoch 5, Loss: 34.8704
Epoch 6, Loss: 28.6299
Epoch 7, Loss: 22.9796
Epoch 8, Loss: 18.5314
Epoch 9, Loss: 15.1249
Epoch 10, Loss: 12.0655
Epoch 11, Loss: 8.7396
Epoch 12, Loss: 5.3330
Epoch 13, Loss: 3.7015
Epoch 14, Loss: 3.7843
Epoch 15, Loss: 4.2347
Epoch 16, Loss: 4.6741
Epoch 17, Loss: 5.0124
Epoch 18, Loss: 5.2163
Epoch 19, Loss: 5.3048
Epoch 20, Loss: 5.3157
Epoch 21, Loss: 5.2776
Epoch 22, Loss: 5.2045
Epoch 23, Loss: 5.1037
Epoch 24, Loss: 4.9808
Epoch 25, Loss: 4.8412
Epoch 26, Loss: 4.6901
Epoch 27, Loss: 4.5323
Epoch 28, Loss: 4.3719
Epoch 29, Loss: 4.2127
Epoch 30, Loss: 4.0582
Epoch 31, Loss: 3.9121
Epoch 32, Loss: 3.7776
Epoch 33, Loss: 3.6580
Epoch 34, Loss: 3.5562
Epoch 35, Loss: 3.4745
Epoch 36, Loss: 3.4143
Epoch 37, Loss: 3.3757
Epoch 38, Loss: 3.3572
Epoch 39, Loss: 3.3557
Epoch 40, Loss: 3.3662
Epoch 41, Loss: 3.3833
Epoch 42, Loss: 3.4016
Epoch 43, Loss: 3.4166
Epoch 44, 

In [17]:
def greedy_decode(logits, blank=0):
    """
    logits: [T, B, C] (CTC 출력 형식)
    returns: List[str], 예: ["hello"]
    """
    argmax_preds = torch.argmax(logits, dim=2)  # [T, B]
    results = []

    for b in range(argmax_preds.size(1)):
        pred = argmax_preds[:, b].tolist()

        # 중복 제거 및 blank 제거 (CTC 규칙)
        decoded = []
        prev = blank
        for p in pred:
            if p != prev and p != blank:
                decoded.append(p)
            prev = p

        result_str = ''.join([idx2char[i] for i in decoded])
        results.append(result_str)

    return results

In [18]:
print("Output shape:", output.shape)  # [B, T, C]
print("Output argmax:", output.argmax(2))  # [B, T]

Output shape: torch.Size([150, 100, 28])
Output argmax: tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        ...,
        [24, 24, 24,  ..., 15, 15, 15],
        [24, 24, 24,  ..., 15, 15, 15],
        [24, 24, 24,  ...,  2,  2,  2]])


In [19]:
import hgtk

def merge_jamos(jamo_seq):
    result = ""
    i = 0
    while i < len(jamo_seq):
        cho = jamo_seq[i]

        if cho not in hgtk.letter.CHO:
            result += cho
            i += 1
            continue

        if i + 1 < len(jamo_seq) and jamo_seq[i + 1] in hgtk.letter.JOONG:
            jung = jamo_seq[i + 1]

            if i + 2 < len(jamo_seq) and jamo_seq[i + 2] in hgtk.letter.JONG:
                jong = jamo_seq[i + 2]

                if i + 3 < len(jamo_seq) and jamo_seq[i + 3] in hgtk.letter.JOONG:
                    try:
                        result += hgtk.letter.compose(cho, jung)
                        i += 2
                        continue
                    except hgtk.exception.NotHangulException:
                        result += cho + jung
                        i += 2
                        continue
                else:
                    try:
                        result += hgtk.letter.compose(cho, jung, jong)
                        i += 3
                        continue
                    except hgtk.exception.NotHangulException:
                        result += hgtk.letter.compose(cho, jung)
                        i += 2
                        continue

            try:
                result += hgtk.letter.compose(cho, jung)
                i += 2
                continue
            except hgtk.exception.NotHangulException:
                result += cho + jung
                i += 2
                continue
        else:
            result += cho
            i += 1

    return result

In [20]:
jamo_index=1
model.eval()
with torch.no_grad():
    output = model(features)  # [1, T, C]
    output = torch.nn.functional.log_softmax(output, dim=2)
    output = output.transpose(0, 1)  # [T, B, C]
    decoded = greedy_decode(output)  # [[ㅎ, ㅏ, ㄴ, ㄱ, ㅜ, ㄱ]]
    jamo_seq = decoded[jamo_index]
    print("자모 시퀀스:", "".join(jamo_seq))
    print("완성형 한글:", merge_jamos(jamo_seq))


자모 시퀀스: ㄷㅗㅇㅘㅈㅝ
완성형 한글: 도와줘


In [26]:
def levenshtein_ops(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = [[0]*(n+1) for _ in range(m+1)]
    op = [[None]*(n+1) for _ in range(m+1)]  # 연산 기록

    for i in range(m+1):
        dp[i][0] = i
        op[i][0] = 'D' if i > 0 else None
    for j in range(n+1):
        dp[0][j] = j
        op[0][j] = 'I' if j > 0 else None

    for i in range(1, m+1):
        for j in range(1, n+1):
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1]
                op[i][j] = 'E'  # 일치
            else:
                del_cost = dp[i-1][j] + 1
                ins_cost = dp[i][j-1] + 1
                sub_cost = dp[i-1][j-1] + 1
                min_cost = min(del_cost, ins_cost, sub_cost)
                dp[i][j] = min_cost
                if min_cost == sub_cost:
                    op[i][j] = 'S'  # 치환
                elif min_cost == del_cost:
                    op[i][j] = 'D'  # 삭제
                else:
                    op[i][j] = 'I'  # 삽입

    # 연산 추적
    i, j = m, n
    subs, ins, dels = 0, 0, 0
    while i > 0 or j > 0:
        if op[i][j] == 'E':
            i -= 1
            j -= 1
        elif op[i][j] == 'S':
            subs += 1
            i -= 1
            j -= 1
        elif op[i][j] == 'D':
            dels += 1
            i -= 1
        elif op[i][j] == 'I':
            ins += 1
            j -= 1

    return dp[m][n], subs, ins, dels

# -------------------------------
# 평가 함수 (WER, CER + 정확도 + 세부연산)
# -------------------------------

def wer_detail(reference, hypothesis):
    ref_words = reference.strip().split()
    hyp_words = hypothesis.strip().split()
    distance, subs, ins, dels = levenshtein_ops(ref_words, hyp_words)
    wer_score = distance / max(len(ref_words), 1)
    accuracy = (len(ref_words) - distance) / max(len(ref_words), 1)
    return wer_score, accuracy, subs, ins, dels

def cer_detail(reference, hypothesis):
    ref_chars = list(reference.strip().replace(" ", ""))
    hyp_chars = list(hypothesis.strip().replace(" ", ""))
    distance, subs, ins, dels = levenshtein_ops(ref_chars, hyp_chars)
    cer_score = distance / max(len(ref_chars), 1)
    accuracy = (len(ref_chars) - distance) / max(len(ref_chars), 1)
    return cer_score, accuracy, subs, ins, dels

# -------------------------------
# 예제 입력 및 출력
# -------------------------------
my_list=["도와줘",
         "구급차 불러",
         "불났어",
         "쓰러졌어",
         "위험해",
         "경찰불러",
         "아파",
         "구조요청",
         "약가져와",
         "응급모드시작"
         ]

index=234
jamo_index=0
for jamo_index in range(10):
  jamo_index*=10
  jamo_seq = decoded[jamo_index]
  hyp_text=merge_jamos(jamo_seq)
  ref_text=my_list[jamo_index//10]

  print("Reference :", ref_text)
  print("Hypothesis:", hyp_text)

  # WER
  wer_score, wer_acc, wer_s, wer_i, wer_d = wer_detail(ref_text, hyp_text)
  print("\n[WER 평가 - 단어 단위]")
  print("WER (오류율)      :", round(wer_score, 3))
  print("정확도 (Accuracy) :", round(wer_acc, 3))
  print("치환(Sub)         :", wer_s)
  print("삽입(Ins)         :", wer_i)
  print("삭제(Del)         :", wer_d)

  # CER
  cer_score, cer_acc, cer_s, cer_i, cer_d = cer_detail(ref_text, hyp_text)
  print("\n[CER 평가 - 문자 단위]")
  print("CER (오류율)      :", round(cer_score, 3))
  print("정확도 (Accuracy) :", round(cer_acc, 3))
  print("치환(Sub)         :", cer_s)
  print("삽입(Ins)         :", cer_i)
  print("삭제(Del)         :", cer_d)

Reference : 도와줘
Hypothesis: 도와줘

[WER 평가 - 단어 단위]
WER (오류율)      : 0.0
정확도 (Accuracy) : 1.0
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오류율)      : 0.0
정확도 (Accuracy) : 1.0
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 0
Reference : 구급차 불러
Hypothesis: 구급차 불러

[WER 평가 - 단어 단위]
WER (오류율)      : 0.0
정확도 (Accuracy) : 1.0
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오류율)      : 0.0
정확도 (Accuracy) : 1.0
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 0
Reference : 불났어
Hypothesis: 불났어

[WER 평가 - 단어 단위]
WER (오류율)      : 0.0
정확도 (Accuracy) : 1.0
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오류율)      : 0.0
정확도 (Accuracy) : 1.0
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 0
Reference : 쓰러졌어
Hypothesis: 쓰러졌어

[WER 평가 - 단어 단위]
WER (오류율)      : 0.0
정확도 (Accuracy) : 1.0
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오